In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [17]:
#conda uninstall -c anaconda py-xgboost
#conda install -c bioconda xgboost=0.6a2
#conda install -c conda-forge xgboost 

In [18]:
#pip install fastparquet
#conda install -c conda-forge python-snappy

In [6]:
train = pd.read_parquet('data_fusion_train.parquet')

## Убираем ненужные теги

In [7]:
train = train[train.category_id != -1]
#train = train[train.category_id != 0]
train = train.drop_duplicates('item_name')

In [8]:
#df = train[:10000]
train = train.drop(['receipt_id','receipt_dayofweek','receipt_time','item_quantity'], axis=1)

## Убираем цифры и дубликаты пробелом, делаем везде lowercase

In [9]:
import re
train['item_name'] = train['item_name'].str.replace('\d+', '')
train['item_name'] = train['item_name'].str.lower()
train['item_name'] = train['item_name'].apply(lambda x: re.sub(r'[^\w]', ' ', str(x)))
train['item_name'] = train['item_name'].apply(lambda x: re.sub(' +', ' ', str(x)))

In [10]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
train['item_name_token'] = train['item_name'].apply(lambda x: word_tokenize(str(x)))

[nltk_data] Downloading package punkt to /Users/ilya/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Выделяем корни во всех словах, чтобы "молоко" и "молок" не воспринимались разными словами

In [11]:
from nltk.stem import SnowballStemmer

snowball = SnowballStemmer(language='russian')
train['item_name_token'] = train['item_name_token'].apply(lambda x: [snowball.stem(y) for y in x])

## Английские наименования также убираем, чтобы они не путали классификатор

In [174]:
#pip install alphabet-detector

In [12]:
from alphabet_detector import AlphabetDetector
ad = AlphabetDetector()

def transf(input):
    output =[]
    for i in range(len(input)):
        if ad.only_alphabet_chars(input[i],'CYRILLIC') == True:
            output.append(input[i])
    return output


train['item_name_token'] = train['item_name_token'].apply(lambda x: transf(x))

In [13]:
train['item_name_token']

1                                             [молок, шт]
3                                [компот, из, изюм, ккал]
4           [макарон, издел, отварн, масл, сливочн, ккал]
17                              [коф, капучин, больш, эд]
40                                [хлеб, на, сыворотк, г]
                                ...                      
45669181                   [напиток, энерг, ред, булл, л]
45681543                               [хеменгуэ, дайкир]
45690702            [пив, светл, халза, об, пл, б, л, шт]
45692298                              [экспресс, педикюр]
45700308           [конфет, харитош, кг, мол, ваф, яшкин]
Name: item_name_token, Length: 48225, dtype: object

In [14]:
train['item_name_token'] = train['item_name_token'].apply(lambda x: ' '.join(x))

In [15]:
train

,item_name,item_price,item_nds_rate,category_id,brands,item_name_token
1,молоко шт,8,2,78,None,молок шт
3,компот из изюма ккал,4,1,71,None,компот из изюм ккал
4,макаронные изделия отварные масло сливочное ккал,4,1,71,None,макарон издел отварн масл сливочн ккал
17,кофе капучино большой эден,12,1,70,None,коф капучин больш эд
40,хлеб на сыворотке г,7,-1,84,None,хлеб на сыворотк г
...,...,...,...,...,...,...
45669181,напиток энерг ред булл л,10,6,83,None,напиток энерг ред булл л
45681543,хеменгуэй дайкири,15,6,0,None,хеменгуэ дайкир
45690702,пиво светлое халзан об пл б л шт,10,6,0,None,пив светл халза об пл б л шт
45692298,экспресс педикюр,15,6,42,None,экспресс педикюр


## В список stopwords добавляем граммы, килограммы, калории и т.д.

In [57]:
from nltk.corpus import stopwords 

nltk.download('words')
#wordsEng = set(nltk.corpus.words.words())

nltk.download('stopwords')
stop = stopwords.words('russian')
stop = stop.extend(['ккал','к','кал','в','асс','шт','ип','ооо','ш','д','а','б','в','е',
            'ё','ж','з','и','й','к','м','н','о','п','р','с','т','у','ф','х','ц','ч','ш','щ','ю','я','a','b','c','d','e','f',
           'd','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','y',
            'z','xl','xxl','new','без','ок','бкк','классическ','на','упак','бел','черн','кдв','зпп','мф','из'
            'порц','особ','по','нов','рецептур','премиум','мк','оа','ая','эдишн','ип'])

[nltk_data] Downloading package words to /Users/ilya/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ilya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [64]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.8)

X_train_sparse = tfidf.fit_transform(train['item_name_token'])
X_train_nsparse = train.drop('item_name', axis=1)

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_train_sparse, train.category_id.values, test_size=0.33, random_state=1)

In [73]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

clf = LinearSVC(verbose=50)
clf.fit(X_train, y_train)

print(cross_val_score(clf, X_train_sparse, train.category_id.values, cv=3))

[LibLinear][LibLinear][LibLinear][LibLinear][0.7633975  0.76703954 0.76726942]


## Итоговый cv-score получается ~76%